In [19]:
#Capacitated facility location problem
#  The capacitated facility location problem is the basis for many practical 
#  optimization problems, where the total demand that each facility may satisfy 
#  is limited. Hence, modeling such problem must take into account both demand 
#  satisfaction and capacity constraints.
#  See https://scipbook.readthedocs.io/en/latest/flp.html
from pulp import *

In [20]:
customers = {1,2,3,4,5}
facilities = {'f1', 'f2','f3'}
demand = {1:80, 2:270, 3:250, 4:160, 5:180}
actcost = {'f1':1000, 'f2':1000, 'f3':1000}
maxam = {'f1':500, 'f2':500, 'f3':500}
transp = {'f1':{1:4, 2:5, 3:6, 4:8, 5:10},
          'f2':{1:6, 2:4, 3:3, 4:5, 5:8},
          'f3':{1:9, 2:7, 3:4, 4:3, 5:4}}
         

In [21]:
prob = LpProblem("FacilityLocation", LpMinimize)

In [22]:
serv_vars = LpVariable.dicts ("Service", 
                             [(i,j) for i in customers 
                                    for j in facilities], 
                             0)
use_vars = LpVariable.dicts ("useLocation", facilities, 0,0, LpBinary)

In [23]:
prob += lpSum(actcost[j]*use_vars[j] for j in facilities) +lpSum (transp[j][i]*serv_vars[(i,j)] for j in facilities for i in customers)
for i in customers:
    prob +=lpSum(serv_vars[(i,j)] for j in facility) == demand[i]
    
for j in facilities:
    prob +=lpSum(serv_vars[(i,j)] for i in customers) <= maxam[j]  * use_vars[j]

for i in customers:   
    for j in facilities:
        prob += serv_vars[(i,j)] <= demand[i]*use_vars[j]
        

In [34]:
prob.solve()
print ("status", LpStatus[prob.status])
print ("Facilities at nodes:", facilities)

status Optimal
Facilities at nodes: {'f3', 'f1', 'f2'}


In [47]:
TOL = 0.00001
for i in facilities:
    if use_vars[i].varValue >TOL:
         print("Establish facility at site", i)
    
for v in prob.variables():
    print (v.name,"=",v.varValue)
    
    

Establish facility at site f3
Establish facility at site f2
Service_(1,_'f1') = 0.0
Service_(1,_'f2') = 80.0
Service_(1,_'f3') = 0.0
Service_(2,_'f1') = 0.0
Service_(2,_'f2') = 270.0
Service_(2,_'f3') = 0.0
Service_(3,_'f1') = 0.0
Service_(3,_'f2') = 150.0
Service_(3,_'f3') = 100.0
Service_(4,_'f1') = 0.0
Service_(4,_'f2') = 0.0
Service_(4,_'f3') = 160.0
Service_(5,_'f1') = 0.0
Service_(5,_'f2') = 0.0
Service_(5,_'f3') = 180.0
useLocation_f1 = 0.0
useLocation_f2 = 1.0
useLocation_f3 = 1.0


In [49]:
print("the cost of production in dollars for one year", value(prob.objective))

the cost of production in dollars for one year 5610.0
